In [1]:
import pandas as pd
import os
import pathlib

In [2]:
datafile = '..\data\schedule.xlsx'

In [4]:
vac = pd.read_excel(datafile,sheet_name='Vacation')
days = pd.read_excel(datafile,sheet_name='Days')
weeks = list(set(days.week))
vac_sched = days.merge(vac,on='Date')

#build list of all potential months call work time units (week or weekend)
#and assign this an index
potential_call = days.callshiftassignment.drop_duplicates()\
    .reset_index().drop('index',axis=1).reset_index()


vac_slots = vac_sched[['week','callshiftassignment','Person']].drop_duplicates()\
    .sort_values(by=['week','Person'])[['callshiftassignment','Person']].drop_duplicates()

vac_slots = vac_slots.merge(potential_call,on='callshiftassignment')

In [5]:
vac_slots = vac_slots.merge(potential_call,on='callshiftassignment')

In [6]:
#write the scheduled vacation weeks into a dict
#this shows which weeks/weekends people cannot work
vac_weeks = {}
for row in vac_slots.index:
    
    data = vac_slots.loc[row]
    if data[1] not in vac_weeks:
        vac_weeks[data[1]]= [data[2]]

    if data[1] in vac_weeks:
        if data[2] not in vac_weeks[data[1]]:
            vac_weeks[data[1]].append(data[2])

#determine which weeks/weekends people can work
#construct a dict of potential call weeks/weekends
not_vac_weeks = {}

for person in vac_weeks:
    for week in potential_call.index:

        #create first entry
        if person not in not_vac_weeks:
            if week not in vac_weeks[person]:
                not_vac_weeks[person] = [week]

        if person in not_vac_weeks:
            if week not in vac_weeks[person]:
                if week not in not_vac_weeks[person]:
                    not_vac_weeks[person].append(week)


In [7]:
not_vac_weeks['Deenah']

[0, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 20, 21, 22]

In [8]:
vac_weeks['Deenah']

[1, 5, 18, 19, 23]

In [9]:
import pulp

In [10]:
residents = list(set(vac.Person))
shifts = list(potential_call.index)

In [11]:
weekends = list(potential_call[potential_call.callshiftassignment.str.contains('weekend')].index)

In [12]:
weekdays = [shift for shift in list(potential_call.index) if shift not in weekends]

In [13]:
x = pulp.LpVariable.dicts(
    'x',
    ((shift, resident)
        for resident in residents
        for shift in shifts),
    cat=pulp.LpBinary)

In [62]:
#grab formulas from Brian Schiller

synth_var_ix = 0
def synth_var():
    global synth_var_ix
    synth_var_ix += 1
    return synth_var_ix

def and_together(x1, x2):
    """
    produce a variable that represents x1 && x2

    That variable can then be used in constraints and the objective func.
    """
    y = pulp.LpVariable('({} AND {})_{}'.format(x1.name, x2.name, synth_var()), cat=pulp.LpBinary)
    model.addConstraint(y >= x1 + x2 - 1, 'constraint{}'.format(synth_var()))
    model.addConstraint(y <= x1, 'constraint{}'.format(synth_var()))
    model.addConstraint(y <= x2, 'constraint{}'.format(synth_var()))
    return y

def or_together(x1, x2):
    y = pulp.LpVariable('({} OR {})_{}'.format(x1.name, x2.name, synth_var()), cat=pulp.LpBinary)
    model.addConstraint(y <= x1 + x2, 'constraint{}'.format(synth_var()))
    model.addConstraint(y >= x1, 'constraint{}'.format(synth_var()))
    model.addConstraint(y >= x2, 'constraint{}'.format(synth_var()))
    return y

def or_all(xs):
    if len(xs) == 2:
        return or_together(*xs)
    if len(xs) == 1:
        return xs[0]
    if len(xs) == 0:
        raise ValueError('Cannot OR together zero variables')
    x1, x2, *x_rest = xs
    y = or_together(x1, x2)
    return or_all([*x_rest, y])

def minimum(*xs, name):
    y = pulp.LpVariable('{}_{}'.format(name, synth_var()), cat=pulp.LpContinuous)
    for x in xs:
        model.addConstraint(y <= x, 'constraint{}'.format(synth_var()))
    return y

def negate(x):
    y = pulp.LpVariable('(NOT {})_{}'.format(x.name, synth_var()), cat=pulp.LpBinary)
    model.addConstraint(y == 1 - x, 'constraint_{}'.format(synth_var()))
    return y

def avg(xs):
    return sum(xs) / len(xs)

def no_two_inpatient_in_a_row(resident):
    sequential_inpatient = []
    for m1, m2 in zip(shifts, shifts[1:]):
        sequential_inpatient.append(
            negate(and_together(
                or_all([x[m1, resident] for shift in shifts]),
                or_all([x[m2, resident] for shift in shifts]),
        )))
    # avg rather than sum below because each of "NOT (Jan inpatient & Feb inpatient)" is a request
    return sum(sequential_inpatient)




In [63]:
model = pulp.LpProblem('Schedule', pulp.LpMaximize)

#constraints for residents
for resident in residents:

    # this is the "no time-turners constraint"
     
    #minumum number of shifts 
    model.addConstraint(
        sum(x[shift, resident] for shift in shifts) == 6,
        F'min {6} shifts per resident {resident}')

    #model.addConstraint(
    #    sum(x[shift, resident] for shift in shifts) <= 7,
    #    F'max {7} shifts per resident {resident}')

    #remove vacation weeks
    for s in vac_weeks[resident]:
        model.addConstraint(
                sum(x[s, resident] for shift in shifts) == 0 ,
                '{} cannot work during {}'.format(resident,s))

    no_two_inpatient_in_a_row(resident)
    
    #no back to back call
    #no_two_shifts_in_a_row(resident)
    #remove vacation weeks
    for s in not_vac_weeks[resident]:
        for i in not_vac_weeks[resident][1:]:
            if i - s == 2:
                model.addConstraint(
                        sum(x[s, resident] for shift in shifts)
                        + sum(x[i, resident] for shift in shifts) <= len(shifts) ,
                        '{} cannot work during {}'.format(resident,s))

    #for w in weekends:
    #    model.addConstraint(
    #        sum(x[w, resident] for shift in shifts) <=  len(shifts),
    #        '{} cannot work more than the {} weekends'.format(resident,w))    

            
    
        #no back to back call
    #no_two_shifts_in_a_row(resident)
    #remove vacation weeks
    #for s in not_vac_weeks[resident]:
    #    for i in weekends:
    #        if i == s:
    #            model.addConstraint(
    #                    sum(x[s, resident] for shift in shifts)
    #                    + sum(x[i, resident] for shift in shifts) <= len(shifts)/3 ,
    #                    '{} cannot work during {}'.format(resident,s))
    

    



for shift in shifts:
    
    #no duplication of shifts
    model.addConstraint(
        sum(x[shift, resident] for resident in residents) == 1,
        F'{1} resident per {shift}')



            #model.addConstraint(
            #        sum(x[minus, resident] for resident in residents)+ sum(x[s, resident] for resident in residents) == 0 ,
            #        '{} cannot work shift {} before {}'.format(resident,s,minus))


In [64]:
# pulp.LpSolverDefault.msg = 1
model.solve()
if pulp.LpStatus[model.status] != 'Optimal':
    raise ValueError(pulp.LpStatus[model.status])

data = [{
    'resident': resident,
    'shift':shift,
    'result': x[shift,resident].varValue}
 for resident in residents for shift in shifts]

result_raw = pd.DataFrame(data)
result_final = result_raw[result_raw.result > 0]

result_final_out = result_final.merge(potential_call,left_on='shift',right_on='index')\
    .merge(days,on='callshiftassignment')

result_final_out.to_excel('..\data\cl_planned_schedule.xlsx',sheet_name='cl_shifts',index=False)

result_final_out.to_clipboard(index=False)

In [65]:
result_final

,resident,shift,result
4,Deenah,4,1.0
9,Deenah,9,1.0
12,Deenah,12,1.0
16,Deenah,16,1.0
20,Deenah,20,1.0
21,Deenah,21,1.0
26,Jess,2,1.0
29,Jess,5,1.0
32,Jess,8,1.0
41,Jess,17,1.0


In [38]:
sched_file = '..\data\cl_planned_schedule_20220617.xlsx'

In [48]:
sched_draft = pd.read_excel(sched_file)
work_shifts = sched_draft[sched_draft.resident != 'x'][['resident','shift']].drop_duplicates()

In [50]:
work_s_dict = {}

for i in work_shifts.index:

    data = work_shifts.loc[i]

    if data[0] not in work_s_dict:
        work_s_dict[data[0]] = [data[1]]

    if data[0] in work_s_dict:
        if data[1] not in work_s_dict[data[0]]:
            work_s_dict[data[0]].append(data[1])



In [51]:
work_s_dict

{'Erin': [1, 9, 11, 14, 17, 22],
 'Jess': [2, 5, 16, 18, 20, 23],
 'Deenah': [3, 6, 8, 12, 15, 21],
 'Paul': [4, 7, 10, 13, 19]}

In [52]:
vac_weeks

{'Deenah': [1, 5, 18, 19, 23],
 'Jess': [1, 0, 13, 14, 15],
 'Erin': [19, 20, 21],
 'Paul': [21, 23, 22]}

In [66]:
bespoke_shifts

{'Erin': [1, 9, 11, 14, 17, 22, 19, 20, 21],
 'Jess': [2, 5, 16, 18, 20, 23, 1, 0, 13, 14, 15],
 'Deenah': [3, 6, 8, 12, 15, 21, 1, 5, 18, 19, 23],
 'Paul': [4, 7, 10, 13, 19, 21, 23, 22]}

In [71]:
#grab formulas from Brian Schiller



synth_var_ix = 0
def synth_var():
    global synth_var_ix
    synth_var_ix += 1
    return synth_var_ix

def and_together_j(x1, x2):
    """
    produce a variable that represents x1 && x2

    That variable can then be used in constraints and the objective func.
    """
    y = pulp.LpVariable('({} AND {})_{}'.format(x1.name, x2.name, synth_var()), cat=pulp.LpBinary)
    model_j.addConstraint(y >= x1 + x2 - 1, 'constraint{}'.format(synth_var()))
    model_j.addConstraint(y <= x1, 'constraint{}'.format(synth_var()))
    model_j.addConstraint(y <= x2, 'constraint{}'.format(synth_var()))
    return y

def or_together_j(x1, x2):
    y = pulp.LpVariable('({} OR {})_{}'.format(x1.name, x2.name, synth_var()), cat=pulp.LpBinary)
    model_j.addConstraint(y <= x1 + x2, 'constraint{}'.format(synth_var()))
    model_j.addConstraint(y >= x1, 'constraint{}'.format(synth_var()))
    model_j.addConstraint(y >= x2, 'constraint{}'.format(synth_var()))
    return y

def or_all_j(xs):
    if len(xs) == 2:
        return or_together_j(*xs)
    if len(xs) == 1:
        return xs[0]
    if len(xs) == 0:
        raise ValueError('Cannot OR together zero variables')
    x1, x2, *x_rest = xs
    y = or_together_j(x1, x2)
    return or_all_j([*x_rest, y])

def minimum_j(*xs, name):
    y = pulp.LpVariable('{}_{}'.format(name, synth_var()), cat=pulp.LpContinuous)
    for x in xs:
        model_j.addConstraint(y <= x, 'constraint{}'.format(synth_var()))
    return y

def negate_j(x):
    y = pulp.LpVariable('(NOT {})_{}'.format(x.name, synth_var()), cat=pulp.LpBinary)
    model_j.addConstraint(y == 1 - x, 'constraint_{}'.format(synth_var()))
    return y

def avg(xs):
    return sum(xs) / len(xs)

def no_two_inpatient_in_a_row_j(resident):
    sequential_inpatient = []
    for m1, m2 in zip(shifts, shifts[1:]):
        sequential_inpatient.append(
            negate_j(and_together_j(
                or_all_j([x_j[m1, resident] for shift in shifts]),
                or_all_j([x_j[m2, resident] for shift in shifts]),
        )))
    # avg rather than sum below because each of "NOT (Jan inpatient & Feb inpatient)" is a request
    return sum(sequential_inpatient)




In [60]:
#combine the dicts to get all days works ed vacation in a list 
# these days can't be used to for jeopardy
bespoke_shifts = {}

for key in work_s_dict.keys():
    bespoke_shifts[key] = work_s_dict[key] + vac_weeks[key]


In [61]:
bespoke_shifts

{'Erin': [1, 9, 11, 14, 17, 22, 19, 20, 21],
 'Jess': [2, 5, 16, 18, 20, 23, 1, 0, 13, 14, 15],
 'Deenah': [3, 6, 8, 12, 15, 21, 1, 5, 18, 19, 23],
 'Paul': [4, 7, 10, 13, 19, 21, 23, 22]}

In [73]:
#assign jeopardy shifts
x_j = pulp.LpVariable.dicts(
    'x',
    ((shift, resident)
        for resident in residents
        for shift in shifts),
    cat=pulp.LpBinary)

In [75]:
model_j = pulp.LpProblem('Schedule_Jeopardy', pulp.LpMaximize)

#constraints for residents
for resident in residents:

    # this is the "no time-turners constraint"
     
    #minumum number of shifts 
    model_j.addConstraint(
        sum(x_j[shift, resident] for shift in shifts) == 6,
        F'min {6} shifts per resident {resident}')

    #model.addConstraint(
    #    sum(x[shift, resident] for shift in shifts) <= 7,
    #    F'max {7} shifts per resident {resident}')

    #remove vacation weeks
    for s in bespoke_shifts[resident]:
        model_j.addConstraint(
                sum(x_j[s, resident] for shift in shifts) == 0 ,
                '{} cannot work during {}'.format(resident,s))

    no_two_inpatient_in_a_row_j(resident)
    
    #no back to back call
    #no_two_shifts_in_a_row(resident)
    #remove vacation weeks
    #for s in not_vac_weeks[resident]:
    #    for i in not_vac_weeks[resident][1:]:
    #        if i - s == 2:
    #            model.addConstraint(
    #                    sum(x[s, resident] for shift in shifts)
    #                    + sum(x[i, resident] for shift in shifts) <= len(shifts) ,
    #                    '{} cannot work during {}'.format(resident,s))

    #for w in weekends:
    #    model.addConstraint(
    #        sum(x[w, resident] for shift in shifts) <=  len(shifts),
    #        '{} cannot work more than the {} weekends'.format(resident,w))    

            
    
        #no back to back call
    #no_two_shifts_in_a_row(resident)
    #remove vacation weeks
    #for s in not_vac_weeks[resident]:
    #    for i in weekends:
    #        if i == s:
    #            model.addConstraint(
    #                    sum(x[s, resident] for shift in shifts)
    #                    + sum(x[i, resident] for shift in shifts) <= len(shifts)/3 ,
    #                    '{} cannot work during {}'.format(resident,s))
    

    



for shift in shifts:
    
    #no duplication of shifts
    model_j.addConstraint(
        sum(x_j[shift, resident] for resident in residents) == 1,
        F'{1} resident per {shift}')



            #model.addConstraint(
            #        sum(x[minus, resident] for resident in residents)+ sum(x[s, resident] for resident in residents) == 0 ,
            #        '{} cannot work shift {} before {}'.format(resident,s,minus))


In [76]:
# pulp.LpSolverDefault.msg = 1
model_j.solve()
if pulp.LpStatus[model.status] != 'Optimal':
    raise ValueError(pulp.LpStatus[model.status])

data_j = [{
    'resident': resident,
    'shift':shift,
    'result': x_j[shift,resident].varValue}
 for resident in residents for shift in shifts]

result_raw_j = pd.DataFrame(data)
result_final_j = result_raw_j[result_raw_j.result > 0]

result_final_out_j = result_final_j.merge(potential_call,left_on='shift',right_on='index')\
    .merge(days,on='callshiftassignment')

result_final_out_j.to_excel('..\data\cl_planned_joepardy.xlsx',sheet_name='cl_j_shifts',index=False)

result_final_out_j.to_clipboard(index=False)

In [83]:
bespoke_shifts['Erin']

[1, 9, 11, 14, 17, 22, 19, 20, 21]

In [84]:
sched_draft_out = sched_draft.merge(result_final_j[['shift','resident']],how='left', on='shift')

In [85]:
sched_draft_out.to_clipboard(index=False)

In [91]:
sched_draft_j = pd.read_excel(sched_file,sheet_name='cl_shifts_jeopardy')


In [92]:
#quality checks

#simulatenous call & jeopardy
sched_draft_j[sched_draft_j.Resident == sched_draft_j.Jeopardy]

,Resident,shift,result,index,callshiftassignment,Date,Weekday,shifttype,week,Vacation,Jeopardy
0,x,0,1,0,weekday0,2022-07-01,6,weekday,27,NaN,x
7,x,2,1,2,weekday1,2022-07-08,6,weekday,28,NaN,x
14,x,4,1,4,weekday2,2022-07-15,6,weekday,29,NaN,x
21,x,6,1,6,weekday3,2022-07-22,6,weekday,30,NaN,x
28,x,8,1,8,weekday4,2022-07-29,6,weekday,31,NaN,x
35,x,10,1,10,weekday5,2022-08-05,6,weekday,32,NaN,x
42,x,12,1,12,weekday7,2022-08-12,6,weekday,33,NaN,x
49,x,14,1,14,weekday8,2022-08-19,6,weekday,34,Jess,x
56,x,16,1,16,weekday9,2022-08-26,6,weekday,35,NaN,x
63,x,18,1,18,weekday10,2022-09-02,6,weekday,36,Deenah,x


In [102]:
shift1 = sched_draft_j[sched_draft_j.Resident != 'x'][['Resident', 'Date']]
shift1.Date = pd.to_datetime(shift1.Date)
shift2 = shift1.copy()

In [110]:
merged = shift1.merge(shift2,on='Resident')
merged['dd'] = merged.Date_y - merged.Date_x

In [118]:
merged[merged.dd == '7 days']

,Resident,Date_x,Date_y,dd
56,Erin,2022-07-30,2022-08-06,7 days
74,Erin,2022-07-31,2022-08-07,7 days
388,Jess,2022-08-22,2022-08-29,7 days
407,Jess,2022-08-23,2022-08-30,7 days
426,Jess,2022-08-24,2022-08-31,7 days
445,Jess,2022-08-25,2022-09-01,7 days
482,Jess,2022-08-30,2022-09-06,7 days
501,Jess,2022-08-31,2022-09-07,7 days
520,Jess,2022-09-01,2022-09-08,7 days
655,Deenah,2022-07-18,2022-07-25,7 days
